In [1]:
#data imputation 
import numpy as np
import pandas as pd
import numpy as np

In [108]:

# # Load the median values for imputation
# sesame_450k_median = pd.read_csv("/tank/projects/vpalagina_hobotnica/hobotnica/sesame_450k_median.csv", index_col=0)

# data = pd.read_parquet("/tank/projects/vpalagina_hobotnica/hobotnica/data with HC/GSE32148.parquet")
# data.index = data["index"]
# data = data.drop(["index"], axis=1)
# imputation_values = sesame_450k_median.loc[data.index]

# # methylation_data_imputed.to_csv('methylation_data_imputed.csv')
# for col in data.columns:
#     data[col] = data[col].fillna(imputation_values["median"])
# data.isna().sum().sum()

np.int64(0)

In [2]:
#Imputation. Here I check the sum of NAs in the samples, if more than half, I delite such sites
#If less than half, I impute from reference(if site is present in it) or calculate the median across samples of
#this exact site and replace with median

import pandas as pd
import os
import numpy as np

# Define the path to the directory containing the Parquet files
directory_path = "/tank/projects/vpalagina_hobotnica/hobotnica/data with HC"

# Load the median values for imputation from the CSV file
sesame_450k_median = pd.read_csv("/tank/projects/vpalagina_hobotnica/hobotnica/sesame_450k_median.csv", index_col=0)

output_directory  = "/tank/projects/vpalagina_hobotnica/hobotnica/data_with_HC_imputed"

# Iterate over each Parquet file in the directory
for filename in os.listdir(directory_path)[20:]:
    file_path = os.path.join(directory_path, filename)
    
    # Load the Parquet file
    data = pd.read_parquet(file_path)
    
    if data.columns[0] != "index":
    # Rename the first column to "index"
        data.rename(columns={data.columns[0]: "index"}, inplace=True)
        
    # Calculate the threshold for maximum allowable NAs in a row (half of the columns)
    na_threshold = len(data.columns)*0.9
    
    # Identify rows with more than 90% columns as NaN and remove them
    data = data[data.isna().sum(axis=1) <= na_threshold]
    
    # Identify common indices between data and median values
    common_indices = data["index"].values
    common_indices = pd.Index(common_indices).intersection(sesame_450k_median.index)
    
    # Align the imputation values with the data index
    imputation_values = sesame_450k_median.loc[common_indices, "median"]
    
    # Set the index to "index" column for alignment
    data.set_index("index", inplace=True)
    
    # Perform imputation for common indices using sesame median values
    for col in data.columns:
        data.loc[common_indices, col] = data.loc[common_indices, col].fillna(imputation_values)
    
   # Handle rows not present in the reference file
    non_common_indices = data.index.difference(common_indices)
    
    # Fill NAs in remaining rows with their row medians
    for index in non_common_indices:
        data.loc[index] = data.loc[index].fillna(data.loc[index].median())
    
    # Reset index to move "index" back to a column
    data.reset_index(inplace=True)
    
    # Save the imputed DataFrame to the output directory
    output_file_path = os.path.join(output_directory, filename)
    data.to_parquet(output_file_path)

print("Imputation complete for all files.")

Imputation complete for all files.


In [5]:
#Removing of NAs values in the column
import os
import pandas as pd

# Define the folder containing the datasets
folder_path = '/tank/projects/vpalagina_hobotnica/hobotnica/data with HC'
output = "/tank/projects/vpalagina_hobotnica/hobotnica/data_with_HC_removed_NA"

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Load the dataset
    df = pd.read_parquet(file_path)
    
    # Check for NA values and remove rows with any NA values
    initial_row_count = len(df)
    df_cleaned = df.dropna()
    final_row_count = len(df_cleaned)
    
    # Save the cleaned dataset back to a new file (optional)
    cleaned_file_path = os.path.join(output, filename)
    df_cleaned.to_parquet(cleaned_file_path)

    print(f"Processed {filename}: Removed {initial_row_count - final_row_count} rows with NA values")

print("All datasets processed.")

Processed GSE56046.parquet: Removed 0 rows with NA values
Processed GSE42861.parquet: Removed 0 rows with NA values
Processed GSE111629.parquet: Removed 869 rows with NA values
Processed GSE32148.parquet: Removed 19576 rows with NA values
Processed GSE87640.parquet: Removed 0 rows with NA values
Processed GSE87648.parquet: Removed 0 rows with NA values
Processed GSE72338.parquet: Removed 33919 rows with NA values
Processed GSE217633.parquet: Removed 0 rows with NA values
Processed GSE214297.parquet: Removed 3272 rows with NA values
Processed GSE134429.parquet: Removed 0 rows with NA values
Processed GSE81961.parquet: Removed 52 rows with NA values
Processed GSE77696.parquet: Removed 10 rows with NA values
Processed GSE67705.parquet: Removed 0 rows with NA values
Processed GSE182991.parquet: Removed 3272 rows with NA values
Processed GSE49909.parquet: Removed 548 rows with NA values
Processed GSE175364.parquet: Removed 446722 rows with NA values
Processed GSE166611.parquet: Removed 2038